# Оптимизация программы на Spark RDD
Как вы уже знаете, правильное использование и переиспольование партишенеров может привести к серьезному ускорению выполнения Spark программы и сокращению объема shuffle. В этом задании вам предстоит оптимизировать модельную задачу.

In [2]:
import os
import sys

SPARK_HOME = "/usr/hdp/current/spark2-client"
PYSPARK_PYTHON = "/opt/conda/envs/dsenv/bin/python"
os.environ["PYSPARK_PYTHON"]= PYSPARK_PYTHON
os.environ["SPARK_HOME"] = SPARK_HOME

PYSPARK_HOME = os.path.join(SPARK_HOME, "python/lib")
sys.path.insert(0, os.path.join(PYSPARK_HOME, "py4j-0.10.7-src.zip"))
sys.path.insert(0, os.path.join(PYSPARK_HOME, "pyspark.zip"))

In [3]:
from pyspark import SparkConf, SparkContext
import random

In [4]:
SPARK_UI_PORT = random.choice(range(10100, 10400))
print(f"Spark UI port is: {SPARK_UI_PORT}")

Spark UI port is: 10107


In [5]:
conf = SparkConf()
conf.set("spark.ui.port", SPARK_UI_PORT)
conf.set("spark.default.parallelism", "10")
sc = SparkContext(conf=conf)

## Задача №1
Выполните следующую ячейку

In [6]:
keys = list(range(20)) * 10
keys_rdd = sc.parallelize(keys)
rdd1 = keys_rdd.map(lambda x: (x, "RDD1"))
rdd2 = keys_rdd.map(lambda x: (x, "RDD2"))

Проверьте число партиций в `rdd1` и `rdd2`. Почему число партиций именно такое?

In [7]:
# Ваш код здесь
rdd1.getNumPartitions(), rdd2.getNumPartitions()

(10, 10)

In [9]:
rdd1.glom().collect(), rdd2.glom().collect()

([[(0, 'RDD1'),
   (1, 'RDD1'),
   (2, 'RDD1'),
   (3, 'RDD1'),
   (4, 'RDD1'),
   (5, 'RDD1'),
   (6, 'RDD1'),
   (7, 'RDD1'),
   (8, 'RDD1'),
   (9, 'RDD1'),
   (10, 'RDD1'),
   (11, 'RDD1'),
   (12, 'RDD1'),
   (13, 'RDD1'),
   (14, 'RDD1'),
   (15, 'RDD1'),
   (16, 'RDD1'),
   (17, 'RDD1'),
   (18, 'RDD1'),
   (19, 'RDD1')],
  [(0, 'RDD1'),
   (1, 'RDD1'),
   (2, 'RDD1'),
   (3, 'RDD1'),
   (4, 'RDD1'),
   (5, 'RDD1'),
   (6, 'RDD1'),
   (7, 'RDD1'),
   (8, 'RDD1'),
   (9, 'RDD1'),
   (10, 'RDD1'),
   (11, 'RDD1'),
   (12, 'RDD1'),
   (13, 'RDD1'),
   (14, 'RDD1'),
   (15, 'RDD1'),
   (16, 'RDD1'),
   (17, 'RDD1'),
   (18, 'RDD1'),
   (19, 'RDD1')],
  [(0, 'RDD1'),
   (1, 'RDD1'),
   (2, 'RDD1'),
   (3, 'RDD1'),
   (4, 'RDD1'),
   (5, 'RDD1'),
   (6, 'RDD1'),
   (7, 'RDD1'),
   (8, 'RDD1'),
   (9, 'RDD1'),
   (10, 'RDD1'),
   (11, 'RDD1'),
   (12, 'RDD1'),
   (13, 'RDD1'),
   (14, 'RDD1'),
   (15, 'RDD1'),
   (16, 'RDD1'),
   (17, 'RDD1'),
   (18, 'RDD1'),
   (19, 'RDD1')],
  [(0, 

Выполните следующую ячейку. Откройте Spark UI и определите какое количество тасков было выполнено? Вспомните, сколько партиций в `rdd1`, объясните разницу.

In [8]:
rdd1.take(10)
# 1 таск

[(0, 'RDD1'),
 (1, 'RDD1'),
 (2, 'RDD1'),
 (3, 'RDD1'),
 (4, 'RDD1'),
 (5, 'RDD1'),
 (6, 'RDD1'),
 (7, 'RDD1'),
 (8, 'RDD1'),
 (9, 'RDD1')]

## Задача №2
Выполните следующую ячейку. Откройте Spark UI и изучите выполненный Job. Посмотрите сколько выполнилось стадий (stage), сколько тасков. Проанализируйте DAG и сопоставьте с трансформациями. Найдите столбцы Shuffle Read и Shuffle Write, вспомните, что они обозначают.

In [10]:
rdd3 = rdd1.join(rdd2)
rdd4 = rdd3.groupByKey()
rdd4.count()

20

## Задача №3
Приступим к оптимизациям. `rdd1` и `rdd2` переиспользуют одну и ту же RDD. Какую оптимизацию стоило бы к ней применить? Попробуйте это сделать и посмотрите, есть ли какие-то изменения в Spark UI?

In [14]:
keys = list(range(20)) * 10
keys_rdd = sc.parallelize(keys).cache()
rdd1 = keys_rdd.map(lambda x: (x, "RDD1"))
rdd2 = keys_rdd.map(lambda x: (x, "RDD2"))
rdd3 = rdd1.join(rdd2)
rdd4 = rdd3.groupByKey()
rdd4.count()

20

In [15]:
rdd1.partitioner == rdd2.partitioner

True

## Задача №4
Как вы уже знаете ко-партиционированные RDD полезны для оптимизации широких трансформаций типа `join`. Сделайте `rdd1` и `rdd2` ко-партиционированными (число партиций 4). Выполните код в ячейке и посмотрите на изменения в Spark UI. Изменился ли DAG? Изменилось ли число стадий, тасков? Как изменился общий объем шаффла? Не забудьте перенести оптимизацию из прошлой задачи.

In [7]:
keys = list(range(20)) * 10
keys_rdd = sc.parallelize(keys).cache()
rdd1 = keys_rdd.map(lambda x: (x, "RDD1"))
rdd2 = keys_rdd.map(lambda x: (x, "RDD2"))
rdd1 = rdd1.partitionBy(4)
rdd2 = rdd2.partitionBy(4)
rdd3 = rdd1.join(rdd2, numPartitions=4)
rdd4 = rdd3.groupByKey()
rdd4.count()

20

## Задача №5
При подобной компоновке партиций появляются лишние шаффлы, хотя общий объем шаффлов снижается. Сравните партишенеры RDD, порождаемой трансформацией `rdd1.join(rdd2)` с партишенерами `rdd1` и `rdd2`. Если есть различия, попробуйте их объяснить. Попытайтесь исправить эту проблему.

In [ ]:
# Сравните партишенеры

In [ ]:
keys = list(range(20)) * 10
keys_rdd = sc.parallelize(keys)
rdd1 = keys_rdd.map(lambda x: (x, "RDD1"))
rdd2 = keys_rdd.map(lambda x: (x, "RDD2"))
rdd3 = rdd1.join(rdd2)
rdd4 = rdd3.groupByKey()
rdd4.count()

## Задача №6
Лишняя стадия пропала, но у нас остался еще один лишний шаффл. Попробуйте сами понять, в чем проблема, изучив партишенеры различных RDD. В результате всех оптимизаций вы должны получить джоб, с 3 стадиями, числом тасков меньшим, чем в самом первом запуске без оптимизаций и минимальным общим объемом шаффла.

In [ ]:
keys = list(range(20)) * 10
keys_rdd = sc.parallelize(keys)
rdd1 = keys_rdd.map(lambda x: (x, "RDD1"))
rdd2 = keys_rdd.map(lambda x: (x, "RDD2"))
rdd3 = rdd1.join(rdd2)
rdd4 = rdd3.groupByKey()
rdd4.count()

In [55]:
sc.stop()